In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
from matplotlib.pyplot import *
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
from torchvision import transforms
from torchvision import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error, log_loss
from functools import partial
from dataclasses import dataclass
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
import os
from PIL import Image
import torchvision.transforms.functional as TF
import time
from sklearn.svm import SVC
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
cuda = torch.cuda.is_available()
cuda

True

In [ ]:
from google.colab import files
files.upload()

In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d yash612/covidnet-mini-and-gan-enerated-chest-xray

In [ ]:
! unzip covidnet-mini-and-gan-enerated-chest-xray.zip -d data

In [ ]:
trans_apply=transforms.Compose([transforms.Resize((384,384)),transforms.ToTensor()])
train_dataset=datasets.ImageFolder('data/chest_xray/chest_xray/train',transform=trans_apply)
val_dataset=datasets.ImageFolder('data/chest_xray/chest_xray/val',transform=trans_apply)
test_dataset=datasets.ImageFolder('data/chest_xray/chest_xray/test',transform=trans_apply)

In [ ]:
num_workers = 8 if cuda else 0 
    
# Training data
train_loader_args = dict(shuffle=True, batch_size=8, num_workers=num_workers) if cuda\
                    else dict(shuffle=True, batch_size=16)
train_loader = data.DataLoader(train_dataset, **train_loader_args)

# Validation data
val_loader_args = dict(shuffle=True, batch_size=8, num_workers=num_workers) if cuda\
                    else dict(shuffle=True, batch_size=16)
val_loader = data.DataLoader(val_dataset, **val_loader_args)

# Testing data
test_loader_args = dict(shuffle=False, num_workers=num_workers) if cuda\
                    else dict(shuffle=False,drop_last=True)
test_loader = data.DataLoader(test_dataset, **test_loader_args)


In [ ]:
class Network_new(nn.Module):
    def __init__(self, num_classes):
        super(Network_new, self).__init__()
        self.layer1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.Dropout(0.3),)
        self.layer2 = nn.Sequential(nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(64), nn.ReLU(), nn.Dropout(0.3),)
        self.layer3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer4 = nn.Sequential(nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.Dropout(0.3),)
        self.layer5 = nn.Sequential(nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(128), nn.ReLU(), nn.Dropout(0.3),)
        self.layer6 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer7 = nn.Sequential(nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.Dropout(0.3),)
        self.layer8 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.Dropout(0.3),)
        self.layer9 = nn.Sequential(nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.Dropout(0.3),)
        self.layer10 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer11 = nn.Sequential(nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.Dropout(0.3),)
        self.layer12 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.Dropout(0.3),)
        self.layer13 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.Dropout(0.3),)
        self.layer14 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.layer15 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.Dropout(0.3),)
        self.layer16 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.Dropout(0.3),)
        self.layer17 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(512), nn.ReLU(), nn.Dropout(0.3),)
        self.layer18 = nn.MaxPool2d(kernel_size=2, stride=2)
        #self.layer1 = nn.Sequential(nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1), nn.BatchNorm2d(256), nn.ReLU(), nn.Dropout(0.3),)
        
        #self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        #self.classifier = nn.Sequential(nn.Linear(512*7*7, 4096), nn.ReLU(True), nn.Dropout(), nn.Linear(4096, 4096), nn.ReLU(True), nn.Dropout(),)
        self.avgpool = nn.AdaptiveAvgPool2d((4, 4))
        self.classifier = nn.Sequential(nn.Linear(512*4*4, 4096), nn.ReLU(True), nn.Dropout(), nn.Linear(4096, 4096), nn.ReLU(True), nn.Dropout(),)
        #nn.Linear(4096, num_classes),
        self.linear_label_last = nn.Linear(4096, num_classes, bias=False)
        # For creating the embedding to be passed into the Center Loss criterion
        
    def forward(self, x):
        output = self.layer1(x)
        output = self.layer2(output)
        output = self.layer3(output)
        output = self.layer4(output)
        output = self.layer5(output)
        output = self.layer6(output)
        output = self.layer7(output)
        output = self.layer8(output)
        output = self.layer9(output)
        output = self.layer10(output)
        output = self.layer11(output)
        output = self.layer12(output)
        output = self.layer13(output)
        output = self.layer14(output)
        output = self.layer15(output)
        output = self.layer16(output)
        output = self.layer17(output)
        output = self.layer18(output)

      
        #print(output.shape)
        #output = F.avg_pool2d(output, [output.size(2), output.size(3)], stride=1)
        output = self.avgpool(output)
        #print(output.shape)
        #output = output.reshape(output.shape[0], output.shape[1])
        output = output.reshape(output.shape[0], -1)
        #print(output.shape)
        
        output = self.classifier(output)
        embedding = output
        label_output = self.linear_label_last(output)
        #label_output = self.linear_label(output)
        #label_output = label_output/torch.norm(self.linear_label.weight, dim=1)
        
        # Create the feature embedding for the Center Loss
        #closs_output = self.linear_closs(output)
        #closs_output = self.relu_closs(closs_output)

        #return closs_output, label_output
        return label_output, embedding

def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)


In [ ]:
model = Network_new(3)
criterion = nn.CrossEntropyLoss()
learningRate = 0.005
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate, weight_decay=0.0005, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.9)
device = torch.device("cuda" if cuda else "cpu")
model.to(device)

In [ ]:
def train(model, data_loader, test_loader, task='Classification'):
    model.train()

    for epoch in range(numEpochs):
        avg_loss = 0.0
        for batch_num, (feats, labels) in enumerate(data_loader):
            feats, labels = feats.to(device), labels.to(device)
            #print(feats.device)
            optimizer.zero_grad()
            outputs = model(feats)[0]

            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()

            if batch_num % 50 == 49:
                print('Epoch: {}\tBatch: {}\tAvg-Loss: {:.4f}'.format(epoch+1, batch_num+1, avg_loss/50))
                avg_loss = 0.0    
            
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
        if task == 'Classification':
            val_loss, val_acc = test_classify(model, test_loader)
            train_loss, train_acc = test_classify(model, data_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))
            #test_verify(model, threshold=0.4)
            #test_verify_test(model, threshold=0.4, epoch=epoch)
        scheduler.step()
        #if task == 'Verification':
         # test_verify(model, threshold=0.8, text=textloc)


def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0
    #with torch.no_grad():
    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)[0]
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total

def testing(model, test_loader, criterion):
    with torch.no_grad():
        model.eval()
        running_loss = 0.0
        total_predictions = 0.0
        correct_predictions = 0.0
        model.cuda()
        for batch_idx, (data, target) in enumerate(test_loader):   
            data = data.cuda()
            target = target.cuda()

            outputs,embeddings = model(data)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += target.size(0)
            correct_predictions += (predicted == target).sum().item()

            loss = criterion(outputs, target).detach()
            running_loss += loss.item()

        acc = (correct_predictions/total_predictions)*100.0
        running_loss /= len(test_loader)
        
        print('Test Loss: ', running_loss)
        print('Test Accuracy: ', acc, '%')
        return running_loss, acc


In [ ]:
n_epochs = 30
training_loss = []
testing_loss = []
testing_acc = []

for i in range(n_epochs):
    print("Epoch No.",i+1)
    train_loss = train(model, train_loader, criterion, optimizer)
    test_loss, test_acc = testing(model, test_loader, criterion)
    training_loss.append(train_loss)
    testing_loss.append(test_loss)
    testing_acc.append(test_acc)
    scheduler.step()